In [1]:
#!/usr/bin/env python3
#
# FileSender www.filesender.org
#
# Copyright (c) 2009-2019, AARNet, Belnet, HEAnet, SURFnet, UNINETT
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# *   Redistributions of source code must retain the above copyright
#     notice, this list of conditions and the following disclaimer.
# *   Redistributions in binary form must reproduce the above copyright
#     notice, this list of conditions and the following disclaimer in the
#     documentation and/or other materials provided with the distribution.
# *   Neither the name of AARNet, Belnet, HEAnet, SURFnet and UNINETT nor the
#     names of its contributors may be used to endorse or promote products
#     derived from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.


In [2]:
import argparse
import requests
import time
from collections.abc import Iterable
from collections.abc import MutableMapping
import hmac
import hashlib
import urllib3
import os
import json
import configparser
from os.path import expanduser


In [3]:
from multiprocessing import Pool
from functools import partial
import time

In [4]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [5]:
def flatten(d, parent_key=''):
    items = []
    for k, v in d.items():
        new_key = parent_key + '[' + k + ']' if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten(v, new_key).items())
        else:
            items.append(new_key+'='+v)
    items.sort()
    return items



In [6]:

def call(method, path, data, content=None, rawContent=None, options={}):
    data['remote_user'] = username
    data['timestamp'] = str(round(time.time()))
    flatdata = flatten(data)
    signed = bytes(method+'&'+base_url.replace('https://', '',
                   1).replace('http://', '', 1)+path+'?'+('&'.join(flatten(data))), 'ascii')

    content_type = options['Content-Type'] if 'Content-Type' in options else 'application/json'

    inputcontent = None
    if content is not None and content_type == 'application/json':
        inputcontent = json.dumps(content, separators=(',', ':'))
        signed += bytes('&'+inputcontent, 'ascii')
    elif rawContent is not None:
        inputcontent = rawContent
        signed += bytes('&', 'ascii')
        signed += inputcontent

    # print(signed)
    bkey = bytearray()
    bkey.extend(map(ord, apikey))
    data['signature'] = hmac.new(bkey, signed, hashlib.sha1).hexdigest()

    url = base_url+path+'?'+('&'.join(flatten(data)))
    headers = {
        "Accept": "application/json",
        "Content-Type": content_type
    }
    response = None
    if method == "get":
        response = requests.get(url, verify=not insecure, headers=headers)
    elif method == "post":
        response = requests.post(
            url, data=inputcontent, verify=not insecure, headers=headers)
    elif method == "put":
        response = requests.put(url, data=inputcontent,
                                verify=not insecure, headers=headers)
    elif method == "delete":
        response = requests.delete(url, verify=not insecure, headers=headers)

    if response is None:
        raise Exception('Client error')

    code = response.status_code
    # print(url)
    # print(inputcontent)
    # print(code)
    # print(response.text)

    if code != 200:
        if method != 'post' or code != 201:
            raise Exception('Http error '+str(code)+' '+response.text)

    if response.text == "":
        raise Exception('Http error '+str(code)+' Empty response')

    if method != 'post':
        return response.json()

    r = {}
    r['location'] = response.headers['Location']
    r['created'] = response.json()
    return r


In [7]:

def postTransfer(user_id, files, recipients, subject=None, message=None, expires=None, options=[]):

    if expires is None:
        expires = round(time.time()) + (default_transfer_days_valid*24*3600)

    to = [x.strip() for x in recipients.split(',')]

    return call(
        'post',
        '/transfer',
        {},
        {
            'from': user_id,
            'files': files,
            'recipients': to,
            'subject': subject,
            'message': message,
            'expires': expires,
            'aup_checked': 1,
            'options': options
        },
        None,
        {}
    )


In [8]:
def putChunk(t, f, chunk, offset):
    return call(
        'put',
        '/file/'+str(f['id'])+'/chunk/'+str(offset),
        {'key': f['uid'], 'roundtriptoken': t['roundtriptoken']},
        None,
        chunk,
        {'Content-Type': 'application/octet-stream'}
    )



In [9]:
def fileComplete(t,f):
    return call(
        'put',
        '/file/'+str(f['id']),
        { 'key': f['uid'], 'roundtriptoken': t['roundtriptoken'] },
        { 'complete': True },
        None,
        {}
    )


In [10]:


def deleteTransfer(transfer):
    return call(
        'delete',
        '/transfer/'+str(transfer['id']),
        {'key': transfer['files'][0]['uid']},
        None,
        None,
        {}
    )

In [11]:

def transferComplete(transfer):
    return call(
        'put',
        '/transfer/'+str(transfer['id']),
        {'key': transfer['files'][0]['uid']},
        {'complete': True},
        None,
        {}
    )

In [24]:
def upload_file( fileobject, transferData, filesData, upload_chunk_size, debug):
    fname = fileobject["name"]
    fsize = fileobject["size"]
    fstring = f"{fname}:{fsize}"
    fpath = filesData[fstring]["path"]

    try:
        # putChunks
        if debug:
            print('putChunks: '+fpath)
        with open(fpath, mode='rb', buffering=0) as fin:
            chunk_count = 0
            for offset in range(0, fsize, upload_chunk_size):
                if progress:
                    print('Uploading: '+fpath+' '+str(offset)+'-'+str(min(offset +
                          upload_chunk_size, fsize))+' '+str(round(offset/fsize*100))+'%')
                data = fin.read(upload_chunk_size)
                # print(data)
                putChunk(transferData, fileobject, data, offset)
                if debug:
                    chunk_count += 1
                    print(f"uploaded {chunk_count} chunks")
        # file complete
        if debug:
            print('fileComplete: '+fpath)
        fileComplete(transferData, fileobject)
        if progress:
            print('Uploading: '+fpath+' '+str(size)+' 100%')
    except Exception as e:
        raise(e)


In [25]:

#argv
parser = argparse.ArgumentParser()
parser.add_argument("files", help="path to file(s) to send", nargs='+')
parser.add_argument("-v", "--verbose", action="store_true")
parser.add_argument("-i", "--insecure", action="store_true")
parser.add_argument("-p", "--progress", action="store_true")
parser.add_argument("-s", "--subject")
parser.add_argument("-m", "--message")
requiredNamed = parser.add_argument_group('required named arguments')



In [26]:
filelist = ["/home/paul/Downloads/bowtie-1.3.1-linux-x86_64.zip",
            "/home/paul/Zotero/storage/QS8AEKXV/Gong et al. - 2022 - Analysis and Performance Assessment of the Whole G.pdf",
            "/home/paul/Downloads/I100_issuu_sub.pdf"]

n_procs = min(8, len(filelist))


In [27]:
# settings
base_url = 'https://filesender.aarnet.edu.au/rest.php'
default_transfer_days_valid = 21
username = None
apikey = None
homepath = expanduser("~")

config = configparser.ConfigParser()
config.read(homepath + '/.filesender/filesender.py.ini')
if 'system' in config:
    base_url = config['system'].get(
        'base_url', 'https://filesender.aarnet.edu.au/rest.php')
    default_transfer_days_valid = int(
        config['system'].get('default_transfer_days_valid', 10))
if 'user' in config:
    username = config['user'].get('username')
    apikey = config['user'].get('apikey')





# if we have found these in the config file they become optional arguments
if username is None:
    requiredNamed.add_argument("-u", "--username", required=True)
else:
    parser.add_argument("-u", "--username")

if apikey is None:
    requiredNamed.add_argument("-a", "--apikey", required=True)
else:
    parser.add_argument("-a", "--apikey")

requiredNamed.add_argument("-r", "--recipients", required=True)
args = parser.parse_args("-p -r paul.wang@sahmri.com -v -s subject -m messagebody".split(" ") + filelist)
debug = args.verbose
progress = args.progress
insecure = args.insecure

if args.username is not None:
    username = args.username

if args.apikey is not None:
    apikey = args.apikey



In [28]:
base_url = 'https://filesender.aarnet.edu.au/rest.php'
response = requests.get(base_url+'/info', verify=True)
response.json()


{'url': 'https://filesender.aarnet.edu.au/',
 'name': 'AARNet FileSender',
 'logon_url': 'https://cloudstor.aarnet.edu.au/simplesaml/module.php/core/as_login.php?AuthId=default-sp&ReturnTo=__target__',
 'upload_chunk_size': 12582912}

In [29]:

# configs
try:
    response = requests.get(base_url+'/info', verify=True)
except requests.exceptions.SSLError as exc:
    if not insecure:
        print('Error: the SSL certificate of the server you are connecting to cannot be verified:')
        print(exc)
        print('For more information, please refer to https://www.digicert.com/ssl/. If you are absolutely certain of the identity of the server you are connecting to, you can use the --insecure flag to bypass this warning. Exiting...')
        sys.exit(1)
    elif insecure:
        print('Warning: Error: the SSL certificate of the server you are connecting to cannot be verified:')
        print(exc)
        print('Running with --insecure flag, ignoring warning...')
        response = requests.get(base_url+'/info', verify=False)
upload_chunk_size = response.json()['upload_chunk_size']

if debug:
    print('base_url          : '+base_url)
    print('username          : '+username)
    print('apikey            : '+apikey)
    print('upload_chunk_size : '+str(upload_chunk_size)+' bytes')
    print('recipients        : '+args.recipients)
    print('files             : '+','.join(args.files))
    print('insecure          : '+str(insecure))



base_url          : https://filesender.aarnet.edu.au/rest.php
username          : paul.wang@adelaide.edu.au
apikey            : d1f80e881bb96605a1e903b6647330e7722b43bdd5385a09e3eb97ce9bd4ea19
upload_chunk_size : 12582912 bytes
recipients        : paul.wang@sahmri.com
files             : /home/paul/Downloads/bowtie-1.3.1-linux-x86_64.zip,/home/paul/Zotero/storage/QS8AEKXV/Gong et al. - 2022 - Analysis and Performance Assessment of the Whole G.pdf,/home/paul/Downloads/I100_issuu_sub.pdf
insecure          : False


In [30]:

# postTransfer
if debug:
    print('postTransfer')

files = {}
filesTransfer = []
for f in args.files:
    fn_abs = os.path.abspath(f)
    fn = os.path.basename(fn_abs)
    size = os.path.getsize(fn_abs)

    files[fn+':'+str(size)] = {
        'name': fn,
        'size': size,
        'path': fn_abs
    }
    filesTransfer.append({'name': fn, 'size': size})

troptions = {'get_a_link': 0}


transfer = postTransfer(username,
                        filesTransfer,
                        args.recipients,
                        subject=args.subject,
                        message=args.message,
                        expires=None,
                        options=troptions)['created']
# print(transfer)


postTransfer


In [31]:
transfer

{'id': 3136141,
 'userid': 76641,
 'user_email': 'paul.wang@adelaide.edu.au',
 'subject': 'subject',
 'message': 'messagebody',
 'created': {'raw': 1699402066, 'formatted': '08/11/2023'},
 'expires': {'raw': '1701216458', 'formatted': '29/11/2023'},
 'expiry_date_extension': 0,
 'options': {'get_a_link': False,
  'add_me_to_recipients': False,
  'email_recipient_when_transfer_expires': True,
  'encryption': False},
 'salt': 'SN+YL69fE7nKmKQg8RgdWcZw5kYfOfa9',
 'roundtriptoken': '47a4883b48af99729f9925b0ee646d55',
 'files': [{'id': 21002636,
   'transfer_id': 3136141,
   'uid': 'cd5c8200-dd00-4c95-b796-74b6f21a0ee2',
   'name': 'bowtie-1.3.1-linux-x86_64.zip',
   'size': 52387989,
   'sha1': None,
   'cid': None},
  {'id': 21002641,
   'transfer_id': 3136141,
   'uid': '85740666-ba8b-4ef7-bca0-73d76c65382d',
   'name': 'Gong et al. - 2022 - Analysis and Performance Assessment of the Whole G.pdf',
   'size': 2525461,
   'sha1': None,
   'cid': None},
  {'id': 21002646,
   'transfer_id': 

In [32]:
transfer["files"][0]

{'id': 21002636,
 'transfer_id': 3136141,
 'uid': 'cd5c8200-dd00-4c95-b796-74b6f21a0ee2',
 'name': 'bowtie-1.3.1-linux-x86_64.zip',
 'size': 52387989,
 'sha1': None,
 'cid': None}

In [33]:
files['bowtie-1.3.1-linux-x86_64.zip:52387989']

{'name': 'bowtie-1.3.1-linux-x86_64.zip',
 'size': 52387989,
 'path': '/home/paul/Downloads/bowtie-1.3.1-linux-x86_64.zip'}

In [35]:
filesTransfer

[{'name': 'bowtie-1.3.1-linux-x86_64.zip', 'size': 52387989},
 {'name': 'Gong et al. - 2022 - Analysis and Performance Assessment of the Whole G.pdf',
  'size': 2525461},
 {'name': 'I100_issuu_sub.pdf', 'size': 111724013}]

In [34]:

t0 = time.time()

n_procs = 1

task = partial(upload_file, 
               transferData=transfer, 
               filesData=files,
               upload_chunk_size=upload_chunk_size,
               debug=debug)

pool = Pool(n_procs)
pool.map(task, transfer['files'])
pool.close()

t1 = time.time()

print(t1-t0)





putChunks: /home/paul/Downloads/bowtie-1.3.1-linux-x86_64.zip
Uploading: /home/paul/Downloads/bowtie-1.3.1-linux-x86_64.zip 0-12582912 0%
uploaded 1 chunks
Uploading: /home/paul/Downloads/bowtie-1.3.1-linux-x86_64.zip 12582912-25165824 24%
uploaded 2 chunks
Uploading: /home/paul/Downloads/bowtie-1.3.1-linux-x86_64.zip 25165824-37748736 48%
uploaded 3 chunks
Uploading: /home/paul/Downloads/bowtie-1.3.1-linux-x86_64.zip 37748736-50331648 72%
uploaded 4 chunks
Uploading: /home/paul/Downloads/bowtie-1.3.1-linux-x86_64.zip 50331648-52387989 96%
uploaded 5 chunks
fileComplete: /home/paul/Downloads/bowtie-1.3.1-linux-x86_64.zip
Uploading: /home/paul/Downloads/bowtie-1.3.1-linux-x86_64.zip 111724013 100%
putChunks: /home/paul/Zotero/storage/QS8AEKXV/Gong et al. - 2022 - Analysis and Performance Assessment of the Whole G.pdf
Uploading: /home/paul/Zotero/storage/QS8AEKXV/Gong et al. - 2022 - Analysis and Performance Assessment of the Whole G.pdf 0-2525461 0%
uploaded 1 chunks
fileComplete: /home

In [ ]:
# transferComplete
if debug:
    print('transferComplete')
transferComplete(transfer)
if progress:
    print('Upload Complete')


In [ ]:

try:
    for f in transfer['files']:
        path = files[f['name']+':'+str(f['size'])]['path']
        size = files[f['name']+':'+str(f['size'])]['size']
        # putChunks
        if debug:
            print('putChunks: '+path)
        with open(path, mode='rb', buffering=0) as fin:
            for offset in range(0, size, upload_chunk_size):
                if progress:
                    print('Uploading: '+path+' '+str(offset)+'-'+str(min(offset +
                          upload_chunk_size, size))+' '+str(round(offset/size*100))+'%')
                data = fin.read(upload_chunk_size)
                # print(data)
                putChunk(transfer, f, data, offset)

        # fileComplete
        if debug:
            print('fileComplete: '+path)
        fileComplete(transfer, f)
        if progress:
            print('Uploading: '+path+' '+str(size)+' 100%')

    # transferComplete
    if debug:
        print('transferComplete')
    transferComplete(transfer)
    if progress:
        print('Upload Complete')

except Exception as inst:
    print(type(inst))
    print(inst.args)
    print(inst)

    # deleteTransfer
    if debug:
        print('deleteTransfer')
    deleteTransfer(transfer)
